# Understanding Routing and Scheduling Optimization

## Project Scenario: Delivery Fleet Optimization

In this project, we tackle a common yet complex problem faced by logistics companies worldwide: optimizing the routes and schedules of a delivery fleet. The scenario involves a company that needs to deliver goods to various locations across a city using a fleet of vehicles. Each vehicle has a limited capacity, and each delivery location has a specific demand that must be met within certain working hours.

The challenge is to minimize the total distance traveled by the fleet while ensuring all demands are satisfied and the vehicles do not exceed their capacity or working hours. This involves deciding which vehicle delivers to which location, in what order, and how to schedule these deliveries efficiently.

We use mathematical modeling to represent this problem, incorporating variables for each decision point, constraints to ensure feasibility, and an objective function to minimize total distance. This model is then solved using an optimization solver, providing a practical solution that the company can implement to reduce costs and improve efficiency.

#### Sets and Indices
- $I$: Set of locations, including the depot, indexed by $i, j$.
- $K$: Set of vehicles, indexed by $k$.

#### Parameters
- $d_{ij}$: Distance between locations $i$ and $j$.
- $q_k$: Capacity of vehicle $k$.
- $Q_i$: Demand at location $i$.
- $T$: Maximum working hours for each vehicle.
- $A_{i,k}$: Availability of vehicle $k$ to serve location $i$, where $A_{i,k} = 1$ if vehicle $k$ can serve location $i$, and 0 otherwise.

#### Decision Variables
- $x_{ijk}$: Binary variable that equals 1 if vehicle $k$ travels from location $i$ to $j$, and 0 otherwise.
- $y_{ik}$: Amount of goods delivered to location $i$ by vehicle $k$.

#### Objective Function
$$\text{Minimize} \quad Z = \sum_{i \in I} \sum_{j \in I} \sum_{k \in K} d_{ij} x_{ijk}$$
The objective function aims to minimize the total distance traveled by the fleet, promoting efficiency and reducing operational costs.

#### Constraints
1. **Demand Satisfaction:** Each location's demand must be met exactly once by a vehicle.
$$\sum_{k \in K} y_{ik} = Q_i \quad \forall i \in I$$

2. **Vehicle Capacity:** The total goods delivered by a vehicle cannot exceed its capacity.
$$\sum_{i \in I} y_{ik} \leq q_k \quad \forall k \in K$$

3. **Route Validity:** A vehicle can only travel from location $i$ to $j$ if it is available to serve location $i$.
$$x_{ijk} \leq A_{i,k} \quad \forall i, j \in I, \forall k \in K$$

4. **Working Hours Limitation:** The total distance traveled by a vehicle must not exceed the maximum working hours, considering an average speed or distance covered per hour.
$$\sum_{i \in I} \sum_{j \in I} d_{ij} x_{ijk} \leq T \quad \forall k \in K$$

5. **Binary and Non-negativity Constraints:**
$$x_{ijk} \in \{0, 1\} \quad \forall i, j \in I, \forall k \in K$$
$$y_{ik} \geq 0 \quad \forall i \in I, \forall k \in K$$


In [1]:
from gurobipy import Model, GRB, quicksum

# Example Data Generation
locations = ['Depot', 'Loc1', 'Loc2', 'Loc3', 'Loc4']
demands = {'Loc1': 2, 'Loc2': 3, 'Loc3': 4, 'Loc4': 1}
vehicle_capacity = {0: 5, 1: 6}
distances = {
    ('Depot', 'Loc1'): 10, ('Depot', 'Loc2'): 12, ('Depot', 'Loc3'): 18, ('Depot', 'Loc4'): 20,
    ('Loc1', 'Depot'): 10, ('Loc2', 'Depot'): 12, ('Loc3', 'Depot'): 18, ('Loc4', 'Depot'): 20,
    ('Loc1', 'Loc2'): 8, ('Loc1', 'Loc3'): 9, ('Loc1', 'Loc4'): 14,
    ('Loc2', 'Loc1'): 8, ('Loc2', 'Loc3'): 7, ('Loc2', 'Loc4'): 15,
    ('Loc3', 'Loc1'): 9, ('Loc3', 'Loc2'): 7, ('Loc3', 'Loc4'): 10,
    ('Loc4', 'Loc1'): 14, ('Loc4', 'Loc2'): 15, ('Loc4', 'Loc3'): 10,
}
vehicles = list(vehicle_capacity.keys())
max_hours = 8  # Assuming 8 hours of work per vehicle
t_ij = {key: value / 60 for key, value in distances.items()}  # Simplified travel time

# Model Setup
model = Model("DeliveryRoutingScheduling")

# Decision Variables
x = model.addVars(locations, locations, vehicles, vtype=GRB.BINARY, name="x")
y = model.addVars(locations, vehicles, vtype=GRB.CONTINUOUS, name="y")

# Objective: Minimize total distance
model.setObjective(quicksum(distances[i, j] * x[i, j, k] for i in locations for j in locations if i != j for k in vehicles), GRB.MINIMIZE)

# Constraints
# Each location is visited exactly once by any vehicle
for j in locations[1:]:  # Exclude depot
    model.addConstr(quicksum(x[i, j, k] for i in locations if i != j for k in vehicles) == 1)

# Vehicle capacity
for k in vehicles:
    model.addConstr(quicksum(y[j, k] for j in locations[1:]) <= vehicle_capacity[k])

# Demand satisfaction
for j in locations[1:]:  # Exclude depot
    model.addConstr(quicksum(y[j, k] for k in vehicles) == demands[j])

# Flow conservation
for k in vehicles:
    for j in locations:
        if j != 'Depot':
            model.addConstr(quicksum(x[i, j, k] for i in locations if i != j) == quicksum(x[j, i, k] for i in locations if i != j))

# Working hours constraint
for k in vehicles:
    model.addConstr(quicksum(t_ij[i, j] * x[i, j, k] for i in locations for j in locations if i != j) <= max_hours)

# Solve
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    print(f"Optimal solution found with total distance: {model.ObjVal:.2f}")
    for k in vehicles:
        print(f"\nVehicle {k} route:")
        for i in locations:
            for j in locations:
                if i != j and x[i, j, k].X > 0.5:
                    print(f"{i} to {j} with distance {distances[i, j]}")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 60 columns and 152 nonzeros
Model fingerprint: 0xa8d0d076
Variable types: 10 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [7e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 120.0000000
Presolve removed 8 rows and 20 columns
Presolve time: 0.00s
Presolved: 12 rows, 40 columns, 96 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)

Root relaxation: objective 3.600000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*